In [ ]:
"""
To 


"""

In [117]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from os import sys
sys.path.append("/meshAfterParty/")

In [73]:
import datajoint_utils as du

In [74]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-08 01:14:40,968 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-08 01:14:41,242 - settings - Setting enable_python_native_blobs to True


# Inspecting Decimation

In [10]:
minnie.MultiSomaProofread2()

segment_id segment id for those to be decimated,n_somas the number of somas associated with the neuron from the Allen Institute
864691134947393276,3
864691135012398582,2
864691135012471798,2
864691135012571126,2
864691135012905110,2
864691135065018692,2
864691135065024068,2
864691135065032772,3
864691135065033540,3
864691135081745143,2


In [17]:
minnie.MultiSomaProofread2.proj()

segment_id segment id for those to be decimated
864691134947393276
864691135012398582
864691135012471798
864691135012571126
864691135012905110
864691135065018692
864691135065024068
864691135065032772
864691135065033540
864691135081745143


In [42]:
minnie.MultiSomaProofread2.proj() - minnie.Mesh.proj()

segment_id segment id for those to be decimated


In [129]:
minnie.Decimation() & minnie.MultiSomaProofread2.proj()

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_vertices,n_faces,mesh in-place path to the hdf5 (decimated) mesh file
864691134947393276,0,0.25,1400375,2824255,=BLOB=
864691135012398582,0,0.25,1376990,2794781,=BLOB=
864691135012471798,0,0.25,879119,1793791,=BLOB=
864691135012571126,0,0.25,1405952,2834699,=BLOB=
864691135012905110,0,0.25,816541,1676641,=BLOB=
864691135065018692,0,0.25,927599,1896021,=BLOB=
864691135065024068,0,0.25,1683573,3378174,=BLOB=
864691135065032772,0,0.25,1284918,2610256,=BLOB=
864691135065033540,0,0.25,788810,1618020,=BLOB=
864691135081745143,0,0.25,863788,1773581,=BLOB=


In [460]:
(schema.jobs & "table_name='__baylor_segment_centroid'")#.delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [461]:
schema.external['somas'].delete(delete_external_files=True)

INFO - 2021-01-08 09:23:22,823 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-08 09:23:22,824 - settings - Setting database.user to celiib
INFO - 2021-01-08 09:23:22,825 - settings - Setting database.password to newceliipass
INFO - 2021-01-08 09:23:22,836 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-08 09:23:23,129 - settings - Setting enable_python_native_blobs to True
100%|██████████| 317/317 [00:04<00:00, 76.75it/s]


[]

# Inspecting Soma Extraction

In [459]:
minnie.BaylorSegmentCentroid.delete()

INFO - 2021-01-08 09:21:37,932 - connection - Transaction started


About to delete:
`microns_minnie65_02`.`__baylor_segment_centroid`: 317 items
Proceed? [yes, No]: yes


INFO - 2021-01-08 09:21:39,761 - connection - Transaction committed and closed.


Committed.


In [143]:
segment_id = np.unique(minnie.BaylorSegmentCentroid.fetch("segment_id"))
segment_id

array([864691134947393276, 864691135012398582, 864691135012571126,
       864691135012905110, 864691135065018692, 864691135065024068,
       864691135065032772, 864691135065033540, 864691135081745143,
       864691135081756919, 864691135155574372, 864691135210051520,
       864691135210078912, 864691135274155493, 864691135291777462,
       864691135291788214, 864691135337764326, 864691135337796326,
       864691135341171909, 864691135345134239, 864691135345274271,
       864691135345276831, 864691135345280671, 864691135348374487,
       864691135360133191, 864691135373423816, 864691135373436616,
       864691135382504922, 864691135382996203, 864691135385192277,
       864691135385207893, 864691135394014197, 864691135403709422,
       864691135403794158, 864691135407333321, 864691135428492848,
       864691135440515912, 864691135440585800, 864691135462292893,
       864691135462301853, 864691135467600268, 864691135474614848,
       864691135490263655, 864691135502075061, 864691135510504

In [162]:
len(segment_id)

138

In [453]:
curr_index = 95
curr_segment_id = segment_id[curr_index]
curr_table = minnie.BaylorSegmentCentroid() & dict(segment_id=curr_segment_id)
curr_table

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),soma_index index given to this soma to account for multiple somas in one base semgnet,ver the version number of the materializaiton,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),n_vertices number of vertices,n_faces number of faces,mesh datajoint adapter to get the somas mesh objects,multiplicity the number of somas found for this base segment,sdf sdf width value for the soma,max_side_ratio the maximum of the side length ratios used for check if soma,bbox_volume_ratio ratio of bbox (axis aligned) volume to mesh volume to use for check if soma,max_hole_length euclidean distance of the maximum hole size,run_time the amount of time to run (seconds)
864691135873642126,0,0.25,1,29,234831,159070,24563,15834,31437,=BLOB=,2,0.94,1.041,4.127,49200.409,717.9787
864691135873642126,0,0.25,2,29,217133,165382,24805,14328,28280,=BLOB=,2,0.844,1.113,3.756,981041.679,717.9787


In [454]:
minnie.MultiSomaProofread2 & dict(segment_id=curr_segment_id)

segment_id segment id for those to be decimated,n_somas the number of somas associated with the neuron from the Allen Institute
864691135873642126,3


In [455]:
import neuron_visualizations as nviz
nviz.plot_objects(meshes=du.get_seg_extracted_somas(curr_segment_id),
                 meshes_colors="random")

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [456]:
print(f"number of somas = {np.unique(curr_table.fetch('multiplicity'))[0]}")
du.plot_decimated_mesh_with_somas(curr_segment_id)

number of somas = 2
Segment_id = 864691135873642126
vertices = 1153434, faces= = 2338661


/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [119]:
from importlib import reload
du = reload(du)

INFO - 2021-01-08 01:43:03,778 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-08 01:43:03,779 - settings - Setting database.user to celiib
INFO - 2021-01-08 01:43:03,780 - settings - Setting database.password to newceliipass
INFO - 2021-01-08 01:43:03,783 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-08 01:43:04,067 - settings - Setting enable_python_native_blobs to True


In [457]:
nviz.plot_objects(du.fetch_undecimated_segment_id_mesh(864691135345134239))

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [113]:
import numpy as np
np.unique((minnie.BaylorSegmentCentroid & "segment_id=864691135210051520").fetch("ver"))[0]

29

In [126]:
(schema.jobs & "table_name='__baylor_segment_centroid'") & "status='error'"

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__baylor_segment_centroid,ca15fbed1b5c1061ce8e8a2724dc6c99,error,=BLOB=,UnboundLocalError: local variable 'filtered_soma_list_components' referenced before assignment,=BLOB=,celiib@10.28.0.136,at-node9,29,25110,2021-01-07 19:29:23


In [140]:
((schema.jobs & "table_name='__baylor_segment_centroid'") ).delete()#& "status='error'").fetch("key")#.delete()

In [52]:
print((schema.jobs & "key_hash = '7d2b393cf9636f6c0b4c6af25915ab9f'").fetch1("error_stack"))

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/datajoint/autopopulate.py", line 159, in populate
    make(dict(key))
  File "3_BaylorSegmentCentroid_multi_proofreading.py", line 257, in make
    self.insert(dicts_to_insert,skip_duplicates=True)
  File "/usr/local/lib/python3.6/dist-packages/datajoint/table.py", line 347, in insert
    itertools.chain.from_iterable((v for v in r['values'] if v is not None) for r in rows)))
  File "/usr/local/lib/python3.6/dist-packages/datajoint/connection.py", line 214, in query
    self._execute_query(cursor, query, args, cursor_class, suppress_warnings)
  File "/usr/local/lib/python3.6/dist-packages/datajoint/connection.py", line 196, in _execute_query
    raise translate_query_error(err, query) from None
  File "/usr/local/lib/python3.6/dist-packages/datajoint/connection.py", line 194, in _execute_query
    cursor.execute(query, args)
  File "/usr/local/lib/python3.6/dist-packages/pymysql/cursors.py", line 163, in 

In [75]:
minnie.BaylorSegmentCentroid()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),soma_index index given to this soma to account for multiple somas in one base semgnet,ver the version number of the materializaiton,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),n_vertices number of vertices,n_faces number of faces,mesh datajoint adapter to get the somas mesh objects,multiplicity the number of somas found for this base segment,sdf sdf width value for the soma,max_side_ratio the maximum of the side length ratios used for check if soma,bbox_volume_ratio ratio of bbox (axis aligned) volume to mesh volume to use for check if soma,max_hole_length euclidean distance of the maximum hole size,run_time the amount of time to run (seconds)


In [61]:
import datajoint as dj
m65mat = dj.create_virtual_module('m65mat', 'microns_minnie65_materialization')
m65mat.Materialization.describe()

# version and timestamp of Minnie65 materialization
ver                  : decimal(6,2)                 # materialization version
---
valid=1              : tinyint                      # marks whether the materialization is valid. Defaults to 1.
timestamp=CURRENT_TIMESTAMP : timestamp                    # marks the time at which the materialization service was started



'# version and timestamp of Minnie65 materialization\nver                  : decimal(6,2)                 # materialization version\n---\nvalid=1              : tinyint                      # marks whether the materialization is valid. Defaults to 1.\ntimestamp=CURRENT_TIMESTAMP : timestamp                    # marks the time at which the materialization service was started\n'

In [65]:
schema.external['somas'].delete(delete_external_files=True)

100%|██████████| 1/1 [00:00<00:00, 148.38it/s]


[]

In [62]:
m65mat.Materialization()

ver materialization version,valid marks whether the materialization is valid. Defaults to 1.,timestamp marks the time at which the materialization service was started
0.01,1,2020-07-10 00:19:09
0.08,1,2020-08-01 02:20:00
0.10,1,2020-08-15 14:00:00
0.20,1,2020-09-12 18:14:42
0.30,1,2020-11-02 07:23:42
8.00,1,2020-12-02 03:39:30
9.00,1,2020-12-09 00:51:00
29.00,1,2020-12-22 23:07:22


# Inspecting Decomposition

In [463]:
minnie.Decomposition.delete()

INFO - 2021-01-08 09:23:43,642 - connection - Transaction started


About to delete:
`microns_minnie65_02`.`__decomposition`: 19 items
Proceed? [yes, No]: yes


INFO - 2021-01-08 09:23:45,072 - connection - Transaction committed and closed.


Committed.


In [464]:
schema.external['decomposition'].delete(delete_external_files=True)

100%|██████████| 19/19 [00:00<00:00, 75.73it/s]


[]

In [137]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
(curr_table & "status='error'").delete()

In [136]:
print((curr_table & "key_hash='1c7677a63d052676dc2a5d0e819b9425'").fetch1("key"))

{'segment_id': 864691135589917707, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}


In [468]:
(minnie.schema.jobs & "table_name='__baylor_segment_centroid'")#.delete()